In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F


In [0]:
dbutils.widgets.text("catalogo", "catalog_dev")
dbutils.widgets.text("esquema_source", "silver")
dbutils.widgets.text("esquema_sink", "golden")

In [0]:
catalogo = dbutils.widgets.get("catalogo")
esquema_source = dbutils.widgets.get("esquema_source")
esquema_sink = dbutils.widgets.get("esquema_sink")

In [0]:
df_orders = spark.table(f"{catalogo}.{esquema_source}.orders_transformed")

In [0]:
dim_order = (
    df_orders
    .select(
        col("order_id"),
        col("user_id"),
        col("order_number"),
        col("order_dow"),
        col("order_hour_of_day"),
        col("days_since_prior_order"),
        col("eval_set")
    )
)
dim_order.write.mode("overwrite").saveAsTable(f"{catalogo}.{esquema_sink}.dim_order")

In [0]:
df_order_products_prior = spark.table(f"{catalogo}.{esquema_source}.order_products_prior_transformed")
df_order_products_train = spark.table(f"{catalogo}.{esquema_source}.order_products_train_transformed")

In [0]:
df_order_products_all = (
    df_order_products_prior.withColumn("eval_set_detail", lit("prior"))
    .unionByName(
        df_order_products_train.withColumn("eval_set_detail", lit("train"))
    )
)

In [0]:
fact_order_item = (
    df_order_products_all.alias("op")
    .join(df_orders.alias("o"), col("op.order_id") == col("o.order_id"))
    .select(
        col("op.order_id"),
        col("o.user_id").alias("customer_id"),
        col("o.order_number"),
        col("o.order_dow").alias("date_key"),
        col("o.order_hour_of_day").alias("time_key"),
        col("o.days_since_prior_order"),
        col("op.product_id"),
        col("op.add_to_cart_order"),
        col("op.reordered").alias("reordered_flag"),
        lit(1).alias("quantity"),
        col("o.eval_set"),
        col("op.eval_set_detail")
    )
)
#display(fact_order_item)


In [0]:
fact_order_item.write.mode("overwrite").saveAsTable(f"{catalogo}.{esquema_sink}.fact_order_item")